<a href="https://colab.research.google.com/github/MilciadesGonzalez/TradeLaVirginia/blob/main/ControlHorarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import calendar
import math
from datetime import datetime, timedelta
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
crudo = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.1 - Control de entradas y salidas/ControlHorarios con Python/0 - Crudo/DescargaHorarios.xlsx')
geoCorrecta = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.1 - Control de entradas y salidas/ControlHorarios con Python/1 - DatosExternos/Geo Correcta VF10.xlsx')
supervisor = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.1 - Control de entradas y salidas/ControlHorarios con Python/1 - DatosExternos/Supervisor.xlsx')

In [ ]:
base = crudo[['Cliente','DescripcionCliente','Cargo','CodigoCargo','DescripcionCargo','Recorrido','FechaEntrada','LatitudEntrada','LongitudEntrada','FechaSalida','LatitudSalida','LongitudSalida',
              'LatitudCliente','LongitudCliente','Dia','Plan de Visita']]
base = base[base['Cargo']=='Repositor']
base = pd.merge(base, geoCorrecta, on='Cliente', how='left')
base = pd.merge(base, supervisor, on='CodigoCargo', how='left')
base['LatitudCliente'] = np.where((base.LatitudCliente_y.isna()), base.LatitudCliente_x, base.LatitudCliente_y)
base['LongitudCliente'] = np.where((base.LongitudCliente_y.isna()), base.LongitudCliente_x, base.LongitudCliente_y)
base['LongitudCliente'] = base['LongitudCliente'].astype(float)
base['LatitudCliente'] = base['LatitudCliente'].astype(float)

In [ ]:
base['dif_in'] = 6371000*np.arccos(np.cos(np.radians(90-base.LatitudEntrada))*np.cos(np.radians(90-base.LatitudCliente))+np.sin(np.radians(90-base.LatitudEntrada))*np.sin(np.radians(90-base.LatitudCliente))*np.cos(np.radians(base.LongitudEntrada-base.LongitudCliente)))
base['dif_out'] = 6371000*np.arccos(np.cos(np.radians(90-base.LatitudSalida))*np.cos(np.radians(90-base.LatitudCliente))+np.sin(np.radians(90-base.LatitudSalida))*np.sin(np.radians(90-base.LatitudCliente))*np.cos(np.radians(base.LongitudSalida-base.LongitudCliente)))
base['Ver_In'] = np.where((base.LatitudCliente.isna()),"0Falta XY STD",np.where((base.LatitudEntrada.isna()),"1Sin GPS",np.where((base.dif_in>500),"6Rev +500",np.where((base.dif_in>200),"5Rev +200",
                          np.where((base.dif_in>100),"4Rev +100",np.where((base.dif_in>50),"3Rev +50","2OK"))))))
base['Ver_Out'] = np.where((base.LatitudCliente.isna()),"0Falta XY STD",np.where((base.LatitudSalida.isna()),"1Sin GPS",np.where((base.dif_out>500),"6Rev +500",np.where((base.dif_out>200),"5Rev +200",
                          np.where((base.dif_out>100),"4Rev +100",np.where((base.dif_out>50),"3Rev +50","2OK"))))))
formato = "%H:%M"
h1 = pd.to_datetime(base['FechaEntrada'], formato)
h2 = pd.to_datetime(base['FechaSalida'], formato)

In [ ]:
base['real_Horas'] = (h2 - h1)*24
base[['entrada','salida']] = base['Plan de Visita'].str.split(' - ',expand=True)
base['entrada'] = base['entrada'].fillna(0)
base['salida'] = base['salida'].fillna(0)
base['entrada'] = pd.to_datetime(base['entrada'])
base['salida'] = pd.to_datetime(base['salida'])
base['plan_Horas'] = (base.salida-base.entrada)*24
base['dif_Horas'] = base.real_Horas - base.plan_Horas
base = base.drop(['LatitudCliente_x','LongitudCliente_x','LatitudCliente_y','LongitudCliente_y','entrada','salida'], axis=1)

In [ ]:
base2 = base[['Cliente','DescripcionCliente','Cargo','CodigoCargo','DescripcionCargo','Recorrido','FechaEntrada','LatitudEntrada','LongitudEntrada','LatitudCliente','LongitudCliente',
              'Dia','Plan de Visita','dif_in','Ver_In','real_Horas','plan_Horas','dif_Horas','Supervisor']]
base2['In/Out'] = 'VerIn'
base3 = base[['Cliente','DescripcionCliente','Cargo','CodigoCargo','DescripcionCargo','Recorrido','FechaSalida','LatitudSalida','LongitudSalida','LatitudCliente','LongitudCliente',
              'Dia','Plan de Visita','dif_out','Ver_Out','real_Horas','plan_Horas','dif_Horas','Supervisor']]
base3['In/Out'] = 'VerOut'
base2 = base2.rename(columns={'FechaEntrada':'FechaEntrada/salida','LatitudEntrada':'LatitudEntrada/salida','LongitudEntrada':'LongitudEntrada/salida','dif_in':'dif','Ver_In':'Ver'})
base3 = base3.rename(columns={'FechaSalida':'FechaEntrada/salida','LatitudSalida':'LatitudEntrada/salida','LongitudSalida':'LongitudEntrada/salida','dif_out':'dif','Ver_Out':'Ver'})
baseDinamica = pd.concat([base2, base3])

In [ ]:
base['Cliente'] = '0' + base['Cliente'].astype(str)
baseDinamica['Cliente'] = '0' + baseDinamica['Cliente'].astype(str)

In [ ]:
base.to_excel('base_.xlsx', index=False)
!cp base_.xlsx "/content/drive/Shareddrives/Trade Mktg/Trade/1.1 - Control de entradas y salidas/ControlHorarios con Python"

In [ ]:
baseDinamica.to_excel('baseDinamica_.xlsx', index=False)
!cp baseDinamica_.xlsx "/content/drive/Shareddrives/Trade Mktg/Trade/1.1 - Control de entradas y salidas/ControlHorarios con Python"